**Data Engineering Championship - Problem Statement:**

*Participants need to analyse and create features of the following description:*

    'DATE': create the date from year, month and day of the week 
    'LOW': Lower value of DEP_TIME_BLK
    'HIGH': Higher value of DEP_TIME_BLK
    'TIMESTAMP': create a timestamp with date and lower value of DEP_TIME_BLK
    'WIND_CHILL':  the perceived temperature due to cooling effect of wind blowing   
    'PRCP_SNOW_RATIO': ratio of precipitation and snow
    'PLANE_AGE_AIRLINE_AIRPORT_FLIGHTS_MONTH_RATIO': ratio of plane age and airline and airport flights months.
    'SEAT_DISTRIBUTION': Ratio of seats and in  concurrent flight CONCURRENT_FLIGHTS
    'SEAT_DISTRIBUTION_NORMALISED': normalized values of ratio of seats and in  concurrent flight 

#### **1. Importing Necessary Libraries**

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')